# Evaluate a POMDP Policy

In [1]:
rng = MersenneTwister(1);

In [2]:
using AutomotiveDrivingModels
using AutoViz
using GridInterpolations
using POMDPs
using POMDPToolbox
using AutomotivePOMDPs
using QMDP
using SARSOP
using Parameters
using Reel

09-Aug 16:41:35:WARNING:root:replacing docs for 'AutomotivePOMDPs.scene_to_states :: Tuple{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.
09-Aug 16:41:40:WARNING:root:replacing docs for 'AutomotivePOMDPs.states_to_scene :: Tuple{Dict{Int64,AutomotivePOMDPs.SingleOCState},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.


In [3]:
# include("../src/explicit_pomdps/single_crosswalk/decomposition.jl")

## Define Environment

In [4]:
params = CrosswalkParams(ped_rate=0.1)
env = CrosswalkEnv(params)

AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.1, 2.0, 10.0))

## Solve Policy in Discrete, Single Agent Environment

In [154]:
dpomdp = SingleOCPOMDP(env=env, ΔT = 0.5, p_birth = 0.3, pos_res = 1.0, vel_res=1.0)
solver = QMDPSolver(max_iterations=100, tolerance=1e-3, verbose=true)
dpolicy = solve(solver, dpomdp);

[Iteration 1   ] residual:          1 | iteration runtime:   4672.459 ms, (      4.67 s total)
[Iteration 2   ] residual:       0.95 | iteration runtime:   5048.175 ms, (      9.72 s total)
[Iteration 3   ] residual:      0.903 | iteration runtime:   4935.603 ms, (      14.7 s total)
[Iteration 4   ] residual:      0.857 | iteration runtime:   4912.426 ms, (      19.6 s total)
[Iteration 5   ] residual:      0.811 | iteration runtime:   4837.065 ms, (      24.4 s total)
[Iteration 6   ] residual:      0.762 | iteration runtime:   4809.234 ms, (      29.2 s total)
[Iteration 7   ] residual:      0.707 | iteration runtime:   4874.705 ms, (      34.1 s total)
[Iteration 8   ] residual:      0.647 | iteration runtime:   5056.031 ms, (      39.1 s total)
[Iteration 9   ] residual:      0.519 | iteration runtime:   4767.162 ms, (      43.9 s total)
[Iteration 10  ] residual:      0.466 | iteration runtime:   4730.624 ms, (      48.6 s total)
[Iteration 11  ] residual:       0.41 | iteration 

In [155]:
using JLD
JLD.save("policy.jld", "policy", dpolicy)

In [156]:
using JLD
dpolicy = load("policy.jld")["policy"];

In [157]:
#TODO simulate in the planning environment
sim = HistoryRecorder(rng=rng)
dpomdp.p_birth = 0.
up = SingleOCUpdater(dpomdp)
hist = simulate(sim, dpomdp, dpolicy, up, AutomotivePOMDPs.initial_distribution_no_ped(dpomdp));

In [158]:
duration, fps, render_rec = animate_hist(dpomdp, hist)
speed_factor = 1
film = roll(render_rec, fps = fps*speed_factor, duration = duration/speed_factor)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpiRDr0m", 0x0000000000000009, 2.0, nothing)

## Set up High Fidelity Evaluation Environment

In [159]:
include("../src/eval-crosswalk/config.jl")
include("../src/eval-crosswalk/ego_control.jl")
include("../src/eval-crosswalk/sensor.jl")
include("../src/eval-crosswalk/policy.jl")
include("../src/eval-crosswalk/state_estimation.jl")
include("../src/eval-crosswalk/simulation.jl")
include("../src/eval-crosswalk/render_helpers.jl")

In [160]:
config = EvalConfig(time_out = 300, n_episodes = 500)

EvalConfig 
	 Sim step (s) 0.10 
	 N episodes  500 
	 Time out  300 
	 Time out  300 


In [161]:
cam = StaticCamera(VecE2(25, 0.), 20.);

In [217]:
dpomdp.p_birth = 0.1

0.1

In [218]:
sensor = POMDPSensor(pomdp = dpomdp, sensor = SimpleSensor(0.5, 0.5))

POMDPSensor
  pomdp: AutomotivePOMDPs.SingleOCPOMDP
  sensor: SimpleSensor


In [219]:
fieldnames(SimpleSensor)

2-element Array{Symbol,1}:
 :pos_noise
 :vel_noise

In [220]:
policy = QMDPEval(env, dpomdp, dpolicy);

In [221]:
up = MixedUpdater(dpomdp, config.sim_dt);

In [222]:
a0 = -4.
ego0 = initial_ego(env, config.rng)
b0 = Dict{Int64, SingleOCDistribution}()
b0[-1] = initial_state_distribution(dpomdp, ego0.state)
o0 = Dict{Int64, SingleOCObs}()

Dict{Int64,AutomotivePOMDPs.SingleOCState} with 0 entries

In [223]:
update_freq = 5
ego_model = AVDriver(update_freq, 0, env, a0, b0, o0, ego0, sensor, policy, up);

In [224]:
reset_model!(ego_model, ego0)

0.0

In [225]:
overlay = QMDPOverlay(ego_model);

In [226]:
env.params.ped_rate = 0.0
ego0 = initial_ego(env, config.rng)
reset_model!(ego_model, ego0)
models = Dict{Int, DriverModel}()
models[1] = ego_model
scene = initial_scene(models, env, config)
push!(scene, ego0)
nticks = 100
rec = SceneRecord(nticks+1, config.sim_dt)

SceneRecord(nscenes=0)

In [227]:
# execute the simulation
@time simulate!(rec, scene, env, models, nticks, config.rng, config.n_ped, config.callbacks)

  0.513480 seconds (8.17 M allocations: 261.591 MiB, 20.28% gc time)


SceneRecord(nscenes=65)

In [228]:
reset_overlay!(overlay, ego0)
duration, fps, render_rec = animate_record(rec, overlay, 24)
speed_factor = 1
film = roll(render_rec, fps = fps*speed_factor, duration = duration/speed_factor)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpUDfDLA", 0x0000000000000041, 10.0, nothing)

In [78]:
b = models[1].b[-1]
for (i,s) in enumerate(b.it)
    if s.ped == get_off_the_grid(dpomdp)
        println(b.p[i])
        println(i)
    end
end

0.11807258064516152
13


In [118]:
is_observable_fixed(models[1].ego.state, get_off_the_grid(dpomdp), dpomdp.env)

false